In [486]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [487]:
import re #정규 표현식 모듈
import numpy as np
import pandas as pd
import seaborn as sns
import calendar
import matplotlib.cm as cm

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime      # datatime 라이브러리
from matplotlib import pyplot as plt
from matplotlib.colors import Normalize

%matplotlib inline

data = pd.read_csv('/content/drive/MyDrive/Dataton/netflix_titles.csv')

# Data Review

In [488]:
df = pd.DataFrame(data)
df.drop(['show_id','director','cast'],axis=1,inplace=True)

In [489]:
df.isnull().sum()

type              0
title             0
country         831
date_added       10
release_year      0
rating            4
duration          3
listed_in         0
description       0
dtype: int64

# **Data EDA**

## date_added 시간열 분리

In [490]:
df['date_added'] = df['date_added'].apply(lambda x : pd.Timestamp(x))

df['Registered year'] = df['date_added'].apply(lambda x : x.year)
df['Registered month'] = df['date_added'].apply(lambda x : x.month)
df['Registered day'] = df['date_added'].apply(lambda x : x.day)

df.drop(['date_added'],axis=1,inplace=True)

### description 기준 장르 분류

In [491]:
df['Horror'] = df[df['description'].str.contains('[terror][horrible][blood][ghost]',case=False)].listed_in
df['Romance'] = df[df['description'].str.contains('[love][romantic][happy][sad]',case=False)].listed_in
df['Thriller'] = df[df['description'].str.contains('[mysterious][suspense][kill]',case=False)].listed_in
df['SF'] = df[df['description'].str.contains('[monster]*[myth][magic][fantasy]*[dragon][witch]',case=False)].listed_in
df.update(df['Horror'],overwrite=True)
df.update(df['Romance'],overwrite=True)
df.update(df['Thriller'],overwrite=True)
df.update(df['SF'],overwrite=True)
TV_H = df.groupby(['Thriller']).count()
TV_H.iloc[:,[2]]


,country
Thriller,
Action & Adventure,86
"Action & Adventure, Anime Features, Children & Family Movies",3
"Action & Adventure, Anime Features, Classic Movies",1
"Action & Adventure, Anime Features, Horror Movies",1
"Action & Adventure, Anime Features, International Movies",19
...,...
"TV Horror, TV Mysteries, Teen TV Shows",1
"TV Horror, Teen TV Shows",1
"TV Sci-Fi & Fantasy, TV Thrillers",1


## DataFrame split

In [492]:
# TV Show DataFrame 만들기
dataFilter_TV = df['type'] == 'TV Show'
df[dataFilter_TV]

df_TV = df[dataFilter_TV]

# Movie DataFrame 만들기
dataFilter_Movie = df['type'] == 'Movie'
df[dataFilter_Movie]

df_Movie = df[dataFilter_Movie]


## Country null vlaues fill
>TV Show country null vlaues fill

In [493]:
# NaN 채우기
df_TV['country'].fillna("America",inplace =True,limit =79)
df_TV['country'].fillna("Africa",inplace =True,limit =78)
df_TV['country'].fillna("Asia",inplace =True,limit =78)
df_TV['country'].fillna("Europe",inplace =True,limit =78)
df_TV['country'].fillna("Oceania ",inplace =True,limit =78)
# country 결측치 채워졌는지 확인
df_TV['country'].isnull().sum()

0

In [494]:
# 두개 이상의 국가 값 변경
dfTV_strs=df_TV.loc[(df_TV['country'].str.count(','))!=0]

dfTV_strs[0:49]['country'] ='Africa'

dfTV_strs[49:98]['country']='America'

dfTV_strs[98:147]['country']='Asia'

dfTV_strs[147:196]['country']='Europe'

dfTV_strs[196:]['country']='Oceania '

df_TV.update(dfTV_strs,overwrite = True)

> Movie country null vlaues fill


In [495]:
# NaN 채우기
df_Movie['country'].fillna("America",inplace =True,limit =88)
df_Movie['country'].fillna("Africa",inplace =True,limit =88)
df_Movie['country'].fillna("Asia",inplace =True,limit =88)
df_Movie['country'].fillna("Europe",inplace =True,limit =88)
df_Movie['country'].fillna("Oceania ",inplace =True,limit =88)
# country 결측치 채워졌는지 확인
df_Movie['country'].isnull().sum()

0

In [496]:
# 두개 이상의 국가 값 변경
dfMovie_strs=df_Movie.loc[(df_Movie['country'].str.count(','))!=0]

dfMovie_strs[0:215]['country'] ='Africa'

dfMovie_strs[215:430]['country']='America'

dfMovie_strs[430:645]['country']='Asia'

dfMovie_strs[645:860]['country']='Europe'

dfMovie_strs[860:]['country']='Oceania '

df_Movie.update(dfMovie_strs,overwrite = True)

### Rating_band 만들기

> TV Show rating_band
>> * TV-14 : 14세 이상의 청소년들을 위한 프로그램

>> * TV-MA : 17세 이하의 어린이나 청소년들에게 맞지 않는 프로그램

>> * TV-PG : 부모가 정확히 내용을 알고 보여주라는 프로그램

>> * TV-Y : 모든 어린이의 시청이 가능한 프로그램

>> * TV-Y7 : 7세 이상의 어린이들을 대상으로 한 프로그램

>> * TV-G : 모든 연령층이 볼 수 있는 프로그램


In [497]:
df_TV.loc[(df_TV.rating=='TV-14'),'rating'] = 14
df_TV.loc[(df_TV.rating=='TV-MA'),'rating'] = 19
df_TV.loc[(df_TV.rating=='TV-PG'),'rating'] = 0
df_TV.loc[(df_TV.rating=='TV-Y'),'rating'] = 0
df_TV.loc[(df_TV.rating=='TV-Y7'),'rating'] = 0
df_TV.loc[(df_TV.rating=='TV-G'),'rating'] = 0
df_TV.loc[(df_TV.rating=='NR'),'rating'] = 0
df_TV.loc[(df_TV.rating=='TV-Y7-FV'),'rating'] = 0
df_TV.loc[(df_TV.rating=='R'),'rating'] = 0

# TV-PG, TV-Y, TV-Y7, TV-G, NR, TV-Y7-FV, R = 모든 연령을 뜻하는 '0'
# TV-14 = 14세 이상 관람 가능 등급이므로 '14'
# TV-MA = 청소년 관람 불가 등급이므로 '19' 

> Movie rating_band
>> * G : 모든 연령층의 관객들이 볼 수 있는 영화

>> * PG : 어린이 청소년들의 시청여부를 판단해야 하는 영화

>> * PG-13 : 13세 이상의 청소년들이 볼 수 있는 영화

>> * R : 18세 이상의 성인들을 위한 영화

>> * TV-14 : 14세 이상의 청소년들을 위한 영화

>> * TV-G : 모든 연령층이 볼 수 있는 영화

>> * TV-MA : 17세 이하의 어린이나 청소년들에게 맞지 않는 영화

>> * TV-PG : 부모가 정확히 내용을 알고 보여주라는 영화

>> * TV-Y : 모든 어린이의 시청이 가능한 영화

>> * TV-Y7 : 7세 이상의 어린이들을 대상으로 하는 영화

>> * NR : 미성년자 관람이 불가한 영화

>> * NC-17 : 18세 이상의 성인들을 위한 영화(17세 이하의 청소년들은 시청 불가)

>> * TV-Y7-FV : 틴 에이저나 청소년들을 위한 영화

>> * UR : 미성년자 관람이 불가한 영화

In [498]:
df_Movie.loc[(df_Movie.rating=='TV-14'),'rating'] = 14
df_Movie.loc[(df_Movie.rating=='TV-MA'),'rating'] = 19
df_Movie.loc[(df_Movie.rating=='TV-PG'),'rating'] = 0
df_Movie.loc[(df_Movie.rating=='TV-Y'),'rating'] = 0
df_Movie.loc[(df_Movie.rating=='TV-Y7'),'rating'] = 0
df_Movie.loc[(df_Movie.rating=='TV-G'),'rating'] = 0
df_Movie.loc[(df_Movie.rating=='NR'),'rating'] = 19
df_Movie.loc[(df_Movie.rating=='TV-Y7-FV'),'rating'] = 0
df_Movie.loc[(df_Movie.rating=='R'),'rating'] = 19
df_Movie.loc[(df_Movie.rating=='G'),'rating'] = 0
df_Movie.loc[(df_Movie.rating=='PG'),'rating'] = 14
df_Movie.loc[(df_Movie.rating=='PG-13'),'rating'] = 0
df_Movie.loc[(df_Movie.rating=='UR'),'rating'] = 19
df_Movie.loc[(df_Movie.rating=='NC-17'),'rating'] = 19
df_Movie.loc[(df_Movie.rating=='66 min'),'rating'] = 0
df_Movie.loc[(df_Movie.rating=='74 min'),'rating'] = 0
df_Movie.loc[(df_Movie.rating=='84 min'),'rating'] = 0

# TV-PG, TV-Y, TV-Y7, TV-G, TV-Y7-FV, G, PG-13, 66 min, 74 min, 84 min = 모든연령 등급 ='0'
# TV-14, PG = 14세 이상 관람 가능 등급이므로 '14'
# TV-MA, NR, R, UR, NC-17 = 청소년 관람 불가 등급이므로 '19'

### duration 수치형으로 변환
> TV_Show duration 수치형 변환


In [499]:
# duration 종류 확인
df_TV_T = df_TV.groupby(['duration']).count()
df_TV_T.iloc[:,[2]]

,country
duration,
1 Season,1793
10 Seasons,7
11 Seasons,2
12 Seasons,2
13 Seasons,3
15 Seasons,2
17 Seasons,1
2 Seasons,425
3 Seasons,199


In [500]:
def category_duration	(x) :
  if x == "1 Season":
    return 1
  elif x == "2 Seasons" :
    return 2
  elif x == "3 Seasons" :
    return 3
  elif x == "4 Seasons" :
    return 4
  elif x == "5 Seasons" :
    return 5
  elif x == "6 Seasons" :
    return 6 
  elif x == "7 Seasons" :
    return 7
  elif x == "8 Seasons" :
    return 8
  elif x == "9 Seasons" :
    return 9
  elif x == "10 Seasons" :
    return 10
  elif x == "11 Seasons" :
    return 11
  elif x == "12 Seasons" :
    return 12
  elif x == "13 Seasons" :
    return 13
  elif x == "15 Seasons" :
    return 15
  elif x == "17 Seasons" :
    return 17

In [501]:
df_TV['Duration'] = df_TV['duration'].apply(category_duration)
df_TV.drop(['duration'],axis=1,inplace=True)

> Movie duration 수치형 변환

In [502]:
# duration 종류 확인
#df_Movie_T= df_Movie.groupby(['duration']).count()
#df_Movie_T.iloc[:,[2]]

df_Movie['Duration_T']= df_Movie['duration'].str.split(" ",expand=True)[0]
df_Movie.drop(['duration'],axis=1,inplace=True)

# df_Movie 타입 변경 str -> int
df_Movie['Duration_T'] = pd.to_numeric(df_Movie['Duration_T'])

In [503]:
def category_runningtime(x):
  if 0 <= x <60 :
    return 0
  elif 60 <= x <120 :
    return 1
  elif 120 <= x <180 :
    return 2
  elif 180 <= x <240 :
    return 3
  elif 240 <= x <300 :
    return 4
  else :
    return 5

In [504]:
df_Movie['Duration'] = df_Movie['Duration_T'].apply(category_runningtime)
df_Movie.drop(['Duration_T'],axis=1,inplace=True)

In [508]:
df_TV.fillna(0,inplace=True)
df_Movie.fillna(0,inplace=True)

# Data 시각화

,type,title,country,release_year,rating,listed_in,description,Registered year,Registered month,Registered day,Horror,Romance,Thriller,SF,Duration
1,TV Show,Blood & Water,South Africa,2021.0,19,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021.0,9.0,24.0,0,0,"International TV Shows, TV Dramas, TV Mysteries",0,2
2,TV Show,Ganglands,America,2021.0,19,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021.0,9.0,24.0,0,0,0,0,1
3,TV Show,Jailbirds New Orleans,America,2021.0,19,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021.0,9.0,24.0,0,0,0,0,1
4,TV Show,Kota Factory,India,2021.0,19,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021.0,9.0,24.0,0,0,0,0,2
5,TV Show,Midnight Mass,America,2021.0,19,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,2021.0,9.0,24.0,0,0,"TV Dramas, TV Horror, TV Mysteries","TV Dramas, TV Horror, TV Mysteries",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,TV Show,Yu-Gi-Oh! Arc-V,Oceania,2015.0,0,"Anime Series, Kids' TV",Now that he's discovered the Pendulum Summonin...,2018.0,5.0,1.0,0,0,"Anime Series, Kids' TV",0,2
8796,TV Show,Yunus Emre,Turkey,2016.0,0,"International TV Shows, TV Dramas","During the Mongol invasions, Yunus Emre leaves...",2017.0,1.0,17.0,0,0,"International TV Shows, TV Dramas",0,2
8797,TV Show,Zak Storm,Oceania,2016.0,0,Kids' TV,Teen surfer Zak Storm is mysteriously transpor...,2018.0,9.0,13.0,0,0,Kids' TV,Kids' TV,3
8800,TV Show,Zindagi Gulzar Hai,Pakistan,2012.0,0,"International TV Shows, Romantic TV Shows, TV ...","Strong-willed, middle-class Kashaf and carefre...",2016.0,12.0,15.0,"International TV Shows, Romantic TV Shows, TV ...",0,0,0,1
